In [ ]:
import time
import glob
import sys
import os
import numpy as np
import tqdm
from sklearn.model_selection import train_test_split
from tensorflow import keras
from functools import partial
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from scipy.signal import savgol_filter
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow

def data_load(path):
    files = glob.glob(path)
    if not files:
        print("Chck Phath")
        sys.exit()

    _feature=[]
    label=[]
    for address in tqdm.tqdm(files):
        label_ = os.path.basename(address)
        for num in glob.glob(address+'/*'):
            _feature.append(np.load(num))
            label.append([label_])
        print(label_)
    target = np.array(label).flatten()
    return _feature, target.reshape((-1,))

def data_split(feature, target):
    a_set, b_set, a_target, b_target = [], [], [], []
    arr = np.arange(80)
    np.random.seed(18)
    np.random.shuffle(arr)
    arr = np.split(arr,2)
    for batch in range(10):
        batch_feature_array = feature[0+80*batch:80+80*batch]
        batch_target_array = target[0+80*batch:80+80*batch]
        for count in arr[0]:
            a_set.append(batch_feature_array[count])
            a_target.append(batch_target_array[count])
        for count in arr[1]:
            b_set.append(batch_feature_array[count])
            b_target.append(batch_target_array[count])
    return np.array(a_set), np.array(b_set), np.array(a_target), np.array(b_target)


X_train, y_train = data_load('../train_data/*')
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)

X_test, y_test = data_load('../test_data/*')
y_test = encoder.transform(y_test)

X_test,  X_valid, y_test, y_valid = data_split(X_test, y_test)
X_train = np.array(X_train) / 255.0
X_valid = X_valid / 255.0
X_test = X_test / 255.0

y_train = to_categorical(y_train)
y_valid = to_categorical(y_valid)
y_test_raw = y_test.copy()
y_test = to_categorical(y_test)
number_of_classes = y_valid.shape[1]

In [ ]:
tensorflow.random.set_seed(7)

Conv_default = partial(keras.layers.Conv2D,
                        kernel_size=3,padding="SAME")
model = keras.models.Sequential([
    Conv_default(filters=64, kernel_size=3, input_shape=[210, 210, 10]),
    keras.layers.Activation("relu"),
    Conv_default(filters=64),
    keras.layers.Activation("relu"),
    keras.layers.MaxPooling2D(pool_size=2),

    Conv_default(filters=128),
    keras.layers.Activation("relu"),
    Conv_default(filters=128),
    keras.layers.Activation("relu"),
    keras.layers.MaxPooling2D(pool_size=2),

    Conv_default(filters=256),
    keras.layers.Activation("relu"),
    Conv_default(filters=256),
    keras.layers.Activation("relu"),
    keras.layers.MaxPooling2D(pool_size=2),

    Conv_default(filters=512),
    keras.layers.Activation("relu"),
    Conv_default(filters=512),
    keras.layers.Activation("relu"),
    keras.layers.MaxPooling2D(pool_size=2),

    Conv_default(filters=512),
    keras.layers.Activation("relu"),
    Conv_default(filters=512),
    keras.layers.Activation("relu"),
    keras.layers.MaxPooling2D(pool_size=2),

    keras.layers.Flatten(),
    keras.layers.Dense(units=256, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=1),
])
callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
model.compile(loss="mse", optimizer=keras.optimizers.Adam(),
              metrics=["mse", 'mse'])
start = time.time()
history = model.fit(X_train, y_train, epochs=300, callbacks=[callback], validation_data=(X_valid, y_valid))
print('Time :', time.time()-start)
score = model.evaluate(X_test,y_test)
print(score)


model.save("VGG2.h5")
np.save('history.npy',history.history)

In [ ]:

history=np.load('history.npy',allow_pickle='TRUE').item()


In [ ]:
fig, ax = plt.subplots()
sns.set_theme(style="whitegrid")
training_loss = history["loss"]
test_loss = history["val_loss"]
epoch_count = range(1, len(training_loss)+1)

training_acc = history["accuracy"]
test_acc = history["val_accuracy"]

ax.plot(epoch_count, training_loss, "r-")
ax.plot(epoch_count, test_loss, "b-")
plt.legend(["training_loss", "validation_loss"])
ax.set_xlabel("Epoch")
ax.set_ylabel("Loss")
plt.savefig('./figure/loss.png',dpi=300,bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots()
sns.set_theme(style="whitegrid")
ax.plot(epoch_count, training_acc, "r-")
ax.plot(epoch_count, test_acc, "b-")
ax.legend(["training_acc", "validation_acc"])
ax.set_xlabel("Epoch")
ax.set_ylabel("accuracy")
ax.set_ylim(0,1)
plt.savefig('./figure/acc.png',dpi=300,bbox_inches='tight')